# QNLI

## Preparing the Environment

In [ ]:
#Google Colab - Drive Mounting
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
#Install missing library keras-nlp
!pip install -q keras-nlp

In [ ]:
#Import the libraries
import tensorflow as tf
import keras_nlp
from tensorflow import keras
import numpy as np
import os
import re
import string
import random
import pandas as pd

## Data Preprocessing and Parameters Initialization

In [ ]:
#Finetuning Parameters
FINETUNING_LEARNING_RATE = 5e-5
FINETUNING_EPOCHS = 3
FINETUNING_BATCH_SIZE = 32
SEQ_LENGTH = 128

In [ ]:
# Download of the vocabulary from BERT: Bert-uncased
vocab_file = keras.utils.get_file(
    origin="https://storage.googleapis.com/tensorflow/keras-nlp/examples/bert/bert_vocab_uncased.txt",
)
#Initialization of the Word Tokenizer, with a given vocabulary and a sequence length
tokenizer = keras_nlp.tokenizers.WordPieceTokenizer(
    vocabulary=vocab_file, sequence_length=SEQ_LENGTH,
)

In [ ]:
#Definition of the classes dictionary
CLASSES = {
    'not_entailment': 0,
    'entailment': 1,
}

current_path = "/content/gdrive/My Drive/Thesis/Data/GLUE/QNLI"
train_path = os.path.join(current_path, 'train.tsv')
dev_path = os.path.join(current_path, 'dev.tsv')
test_path = os.path.join(current_path, 'test.tsv')

Reference https://github.com/CyberZHG/keras-xlnet/blob/master/demo/GLUE/MNLI/mnli.py

In [ ]:
# Definition of the Class Data using Keras Sequence format
class DataSequence(keras.utils.Sequence):
    '''
    Organize the data into: encoded text/ label 
    Overwrite the Keras.Utils.Sequence Class
    '''
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __len__(self):
        return (len(self.y) + SEQ_LENGTH - 1) // SEQ_LENGTH

    def __getitem__(self, index):
        s = slice(index * SEQ_LENGTH, (index + 1) * SEQ_LENGTH)
        return [item[s] for item in self.x], self.y[s]

def generate_sequence(path):
    '''
    Given a file path, read the file and organize the data from the file
    Return Keras.utils.Sequence object ready to use for training, with the format: tokens/classes
    '''
    tokens, classes = [], []
    df = pd.read_csv(path, sep='\t', error_bad_lines=False)
    for _, row in df.iterrows():
        text_a, text_b, cls = row['question'], row['sentence'], row['label']
        if not isinstance(text_a, str) or not isinstance(text_b, str) or cls not in CLASSES:
            continue
        encoded= (text_a) + '[SEP]' + (text_b) + '[SEP]'
        encoded = tokenizer(encoded)
        tokens.append(encoded)
        classes.append(CLASSES[cls])
    tokens, classes = np.array(tokens), np.array(classes)
    return DataSequence([tokens],classes)

In [ ]:
#Load the training data
train_data = generate_sequence(train_path)
dev_data = generate_sequence(dev_path)

## Load the model and change the head

In [ ]:
# Load the pretrained model and display its structure
model = keras.models.load_model('path_to_the_pretrained_model',compile=False)
model.summary()

In [ ]:
# The model's head is modified for classification


inputs = keras.Input(shape=(SEQ_LENGTH,), dtype=tf.int32)
encoded_tokens = model(inputs)
#HEAD FROM https://keras.io/examples/nlp/text_classification_with_transformer/
x = keras.layers.GlobalAveragePooling1D()(encoded_tokens)
x = keras.layers.Dropout(0.1)(x)
x = keras.layers.Dense(768, activation="tanh")(x)
x = keras.layers.Dropout(0.1)(x)
outputs = keras.layers.Dense(2, activation="softmax")(x)


finetuning_model = keras.models.Model(inputs=inputs, outputs=outputs)
finetuning_model.summary()

In [ ]:
#Create tensorboard callback
logdir = "path_to_save_execution_information" #+ datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

#Compile model 
finetuning_model.compile(
    optimizer=keras.optimizers.Adam(lr=3e-5),
    loss='sparse_categorical_crossentropy',
    metrics=['sparse_categorical_accuracy'],
)

In [ ]:
#Model training
finetuning_model.fit_generator(
    generator=train_data,
    validation_data=dev_data,
    epochs=FINETUNING_EPOCHS,
    callbacks=[tensorboard_callback],
)

In [ ]:
# Add Tokenization layer to the model
inputs = keras.Input(shape=(), dtype=tf.string)
tokens = tokenizer(inputs)
outputs = finetuning_model(tokens)

#Save model
final_model = keras.Model(inputs, outputs)
final_model.save("path_to_save_model")

In [ ]:
#Restore model
restored_model = keras.models.load_model("path_to_save_model", compile=False)

## Testing

In [ ]:
def generate_test(path):
    '''
    Given a file path, read the file and preprocess the data
    Return a numpy array of the encoded test sentences
    '''
    tokens, classes = [], []
    df = pd.read_csv(path, sep='\t', quoting=csv.QUOTE_NONE, error_bad_lines=False)
    print(len(df))
    for _, row in df.iterrows():
        text_a, text_b = row['question'], row['sentence']
        text = (text_a)[:20] + '[SEP]' + (text_b)[:77] + '[SEP]'
        encoded = (text)
        tokens.append(encoded)
    tokens = np.array(tokens)
    return tokens


#Load the test data
test_path = os.path.join(current_path, 'test.tsv')
test_data = generate_test(test_path)

In [ ]:
#Generate predictions
pred = restored_model.predict_generator(test_data, verbose=True,).argmax(axis=-1)

In [ ]:
#Print the number of results
print(len(pred))

In [ ]:
#Format results into dataframe, ready to be uploaded on gluebenchmark.com

REV_CLASSES = {
    0:'not_entailment',
    1:'entailment',
}
results=[]
for p in pred:
  results.append(REV_CLASSES[p])

df_final = pd.DataFrame(results)

df_final.to_csv("QNLI.tsv",sep='\t', encoding='utf-8')

In [ ]:
#Load Tensorboard
%reload_ext tensorboard
%tensorboard --logdir="path_to_save_execution_information"

In [ ]:
#Code to automatically stop the run time for Google Colab
import time
time.sleep(60)
from google.colab import runtime
runtime.unassign()